In [1]:
from data_manipulation import*
from data_preprocessing import*
import pandas as pd
from fm_analycts_parametrs import DEF_STATS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# _df = fm_create_dataframe(path='data/italia_novo_2024.html')
_df = fm_create_dataframe(path='data/SERIEC_2024.html')

In [2]:
df = _df.copy()

In [5]:
df['jg_ar_t_p90'].describe()
df['adef_t_p90'].describe()

count    222.000000
mean      16.819054
std        2.169904
min       11.160000
25%       15.530000
50%       16.600000
75%       17.645000
max       23.890000
Name: adef_t_p90, dtype: float64

In [ ]:
df['rtg_adef2'] = (
    df['rtg_adef']*0.70 + (df['poss_g_p90']/df['adef_t_p90'])*0.20 + np.tanh(df['adef_t_p90']/1)*0.10
).round(2)

In [7]:
df[['nome', 'idade','clube', 'divisao','rtg_jg_ar','rtg_duel','rtg_des','rtg_rec_bola','rtg_adef','rtg_adef2','aval_def']].sort_values(by=['rtg_adef'],ascending=False, ignore_index=True)[:20]

,nome,idade,clube,divisao,rtg_jg_ar,rtg_duel,rtg_des,rtg_rec_bola,rtg_adef,rtg_adef2,aval_def
0,Fabrizio Buschiazzo,27,Cavese,Serie C NOW Girone C,0.66,0.40,0.67,0.66,0.71,0.729208,0.64
1,Brian Silva,21,Pianese,Serie C NOW Girone B,0.63,0.50,0.60,0.56,0.71,0.709049,0.61
2,Christian Pastina,23,Padova,Serie C NOW Girone A,0.60,0.58,0.65,1.02,0.71,0.801806,0.74
3,Mario Noce,24,Legnago,Serie C NOW Girone A,0.64,0.46,0.60,0.86,0.70,0.761087,0.68
4,Andrea Marafini,23,Pineto,Serie C NOW Girone B,0.59,0.54,0.61,0.80,0.70,0.749593,0.67
5,Giovanni Errico,24,Altamura,Serie C NOW Girone C,0.63,0.49,0.68,0.57,0.70,0.704708,0.62
6,Edoardo Grosso,21,Sestri Levante,Serie C NOW Girone B,0.64,0.45,0.62,0.67,0.70,0.723371,0.63
7,Cristian Riggio,27,Taranto,Serie C NOW Girone C,0.62,0.45,0.64,0.77,0.69,0.737313,0.65
8,Alessandro Dalmazzi,29,Lumezzane,Serie C NOW Girone A,0.60,0.46,0.68,0.75,0.69,0.732792,0.65
9,Lorenzo Saporetti,28,Pro Patria,Serie C NOW Girone A,0.60,0.37,0.60,0.73,0.69,0.728867,0.62


In [ ]:
# df['divisao'].unique().tolist()

sorted(df.columns.to_list())

In [ ]:
df['aval_def_adj'] = df['aval_def']*df['coef']
df['rtg_jg_ar_adj'] = df['rtg_jg_ar']*df['coef']
df['rtg_des_adj'] = df['rtg_des']*df['coef']
df['rtg_adef_adj'] = df['rtg_adef']*df['coef']
df['rtg_duel_adj'] = df['rtg_adef']*df['coef']

p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj1'] = (((     (df['rtg_jg_ar_adj']*df['jg_ar_t_p90']*p1) +
                                        df['rtg_duel_adj']*p2 + 
                                        (df['rtg_des_adj']*df['des_t_p90'])*p3 + 
                                        (df['poss_g_p90']/df['adef_t_p90'])*p4+
                                        df['rtg_adef_adj']*p5
                                        ))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj2'] = ((((df['rtg_jg_ar_adj']*p1) + df['rtg_duel_adj']*p2 + (df['rtg_des_adj'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef_adj']*p5))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj3'] = ((((df['rtg_jg_ar']*p1) + df['rtg_duel']*p2 + (df['rtg_des'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef']*p5))).round(2)
df['aval_def_adj4'] = (df['aval_def_adj3']*df['coef']).round(2)

In [ ]:
df_zagueiros = df[df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]
df_zagueiros = df_zagueiros[df_zagueiros['divisao'] == 'Serie C NOW Girone C']
df_zagueiros['aval_def_adj4_pct'] = (df_zagueiros['aval_def_adj4'].rank(pct=True)).round(2)

In [ ]:
df_zagueiros[['nome', 'idade','clube', 'divisao','preco_min', 'posicao_analise', 'aval_def', 'aval_def_adj','aval_def_adj1','aval_def_adj2','aval_def_adj3','aval_def_adj4','aval_def_adj4_pct']].sort_values(by=['aval_def_adj3'],ascending=False, ignore_index=True)

In [ ]:
df_zagueiros['aval_def_adj4'].describe()

In [ ]:
div_med = _df.groupby('divisao')['aval_def'].mean()
div_desv = _df.groupby('divisao')['aval_def'].std()

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

_df_zagueiros['aval_def_adj'] = _df_zagueiros.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_adef_adj'] = _df_zagueiros.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_des_adj'] = _df_zagueiros.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_duel_adj'] = _df_zagueiros.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_jg_ar_adj'] = _df_zagueiros.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

_df_zagueiros['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df_zagueiros['aval_def_adj1'] = (((     (_df_zagueiros['rtg_jg_ar_adj']*_df_zagueiros['jg_ar_t_p90']*p1) +
                                        _df_zagueiros['rtg_duel_adj']*p2 + 
                                        (_df_zagueiros['rtg_des_adj']*_df_zagueiros['des_t_p90'])*p3 + 
                                        (_df_zagueiros['poss_g_p90']/_df_zagueiros['adef_t_p90'])*p4+
                                        _df_zagueiros['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df_zagueiros['aval_def_adj1'] = _df_zagueiros['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df_zagueiros['aval_def_adj2'] = (_df_zagueiros['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

print(_df_zagueiros.groupby('divisao')['rtg_adef'].mean())
print(_df_zagueiros.groupby('divisao')['rtg_adef'].std())

_df_zagueiros[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df_zagueiros[(_df_zagueiros['divisao'].str.contains('Serie C NOW')) & (_df_zagueiros['minutos'] >= 900) & (_df_zagueiros['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'minutos',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def'],ascending=False)

In [ ]:
_df['aval_def_adj'] = _df.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_adef_adj'] = _df.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_des_adj'] = _df.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_duel_adj'] = _df.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_jg_ar_adj'] = _df.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
_df['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df['aval_def_adj1'] = (((     (_df['rtg_jg_ar_adj']*_df['jg_ar_t_p90']*p1) +
                                        _df['rtg_duel_adj']*p2 + 
                                        (_df['rtg_des_adj']*_df['des_t_p90'])*p3 + 
                                        (_df['poss_g_p90']/_df['adef_t_p90'])*p4+
                                        _df['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df['aval_def_adj1'] = _df['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df['aval_def_adj2'] = (_df['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

In [ ]:
_df[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df[(_df['divisao'] == 'Serie A TIM') & (_df['minutos'] >= 900) & (_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def_adj2'],ascending=False)